# Claude Code Colab Bootstrap — DEBUG VERSION

**Hit Runtime → Run all.** Output is logged to `/content/debug_log.txt` for easy sharing.

---

In [ ]:
#@title Setup Logging
import sys
import subprocess
import os
import shutil
import json as json_lib
from datetime import datetime

LOG_FILE = '/content/debug_log.txt'

class TeeLogger:
    def __init__(self, filename):
        self.terminal = sys.stdout
        self.log = open(filename, 'w')
    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)
        self.log.flush()
    def flush(self):
        self.terminal.flush()
        self.log.flush()

sys.stdout = TeeLogger(LOG_FILE)
sys.stderr = TeeLogger(LOG_FILE)

print(f"Debug run started: {datetime.now().isoformat()}")
print(f"Log file: {LOG_FILE}")
print("=" * 60)


In [ ]:
#@title Config + Auth
import os

PROJECT_NAME = "debug-test"
PROJECT_TYPE = "ML/AI Development"
USE_GOOGLE_DRIVE = False

WORKSPACE_PATH = f"/content/claude-workspaces/{PROJECT_NAME}"
CLAUDE_CONFIG_PATH = f"{WORKSPACE_PATH}/.claude"

print(f"Project: {PROJECT_NAME}")
print(f"Workspace: {WORKSPACE_PATH}")

# Auth
print("\n--- Authentication ---")
try:
    from google.colab import userdata
    try:
        token = userdata.get('CLAUDE_OAUTH_TOKEN')
        if token:
            os.environ['CLAUDE_CODE_OAUTH_TOKEN'] = token
            print(f"✓ CLAUDE_OAUTH_TOKEN: {token[:20]}...")
    except Exception as e:
        print(f"No CLAUDE_OAUTH_TOKEN: {e}")
    try:
        key = userdata.get('ANTHROPIC_API_KEY')
        if key:
            os.environ['ANTHROPIC_API_KEY'] = key
            print(f"✓ ANTHROPIC_API_KEY: {key[:10]}...")
    except Exception as e:
        print(f"No ANTHROPIC_API_KEY: {e}")
except ImportError:
    print("Not in Colab")

if not os.environ.get('CLAUDE_CODE_OAUTH_TOKEN') and not os.environ.get('ANTHROPIC_API_KEY'):
    print("⚠️  No auth found. Add to Colab Secrets:")
    print("   CLAUDE_OAUTH_TOKEN (from `claude setup-token`)")
    print("   or ANTHROPIC_API_KEY")


In [ ]:
#@title Install System Dependencies
import subprocess

print("="*60)
print("STEP 1: System Dependencies")
print("="*60)

# Install sandbox dependencies
print("\nInstalling socat and bubblewrap...")
result = subprocess.run(
    "apt-get update -qq && apt-get install -qq -y socat bubblewrap",
    shell=True, capture_output=True, text=True
)
if result.returncode == 0:
    print("✓ Installed")
else:
    print(f"⚠️ Issue: {result.stderr[:200]}")

# Verify
print("\nVerifying:")
for cmd, name in [('socat', 'socat'), ('bwrap', 'bubblewrap')]:
    r = subprocess.run(f"which {cmd}", shell=True, capture_output=True)
    status = "✓" if r.returncode == 0 else "✗"
    print(f"  {status} {name}")


In [ ]:
#@title Install Claude Code + Dev Tools
print("=" * 60)
print("STEP 2: Install Claude Code")
print("=" * 60)

print("\n📦 Installing dev tools...")
subprocess.run("pip install -q black isort", shell=True)
print("✓ black, isort")

print("\n📦 Installing Claude Code...")
result = subprocess.run(
    "curl -fsSL https://claude.ai/install.sh | bash",
    shell=True, capture_output=True, text=True
)
print(f"Return code: {result.returncode}")
# Show last 800 chars of output
output = result.stdout + result.stderr
print(output[-800:] if len(output) > 800 else output)

# Verify
claude_bin = os.path.expanduser("~/.local/bin/claude")
if os.path.exists(claude_bin):
    print(f"\n✓ Claude found: {claude_bin}")
else:
    print(f"\n✗ Claude NOT found at {claude_bin}")


In [ ]:
#@title Step 3: Configure Environment
import json as json_lib

print("=" * 60)
print("STEP 3: Environment Configuration")
print("=" * 60)

# Bashrc
bashrc_path = os.path.expanduser("~/.bashrc")
bashrc_add = '''\n# Claude Code\nexport PATH="$HOME/.local/bin:$HOME/.claude/bin:$PATH"\nexport TERM=xterm-256color\nexport COLORTERM=truecolor\nexport FORCE_COLOR=1\n'''
with open(bashrc_path, 'r') as f:
    content = f.read()
if '.local/bin' not in content:
    with open(bashrc_path, 'a') as f:
        f.write(bashrc_add)
    print("✓ bashrc updated")
else:
    print("✓ bashrc already configured")

# Set for this process
os.environ['PATH'] = os.path.expanduser('~/.local/bin') + ':' + os.environ['PATH']
os.environ['TERM'] = 'xterm-256color'
os.environ['COLORTERM'] = 'truecolor'
os.environ['FORCE_COLOR'] = '1'
print("✓ Env vars set")

# Workspace
os.makedirs(WORKSPACE_PATH, exist_ok=True)
os.makedirs(CLAUDE_CONFIG_PATH, exist_ok=True)
print(f"✓ Workspace: {WORKSPACE_PATH}")

# Symlink
home_claude = os.path.expanduser("~/.claude")
if os.path.islink(home_claude):
    os.unlink(home_claude)
elif os.path.exists(home_claude):
    shutil.rmtree(home_claude) if os.path.isdir(home_claude) else os.unlink(home_claude)
os.symlink(CLAUDE_CONFIG_PATH, home_claude)
print(f"✓ Symlink: ~/.claude → {CLAUDE_CONFIG_PATH}")

# Write credentials if we have OAuth token
oauth_token = os.environ.get('CLAUDE_CODE_OAUTH_TOKEN')
if oauth_token:
    creds_path = f"{CLAUDE_CONFIG_PATH}/.credentials.json"
    creds = {'claudeAiOauth': {'accessToken': oauth_token}}
    with open(creds_path, 'w') as f:
        json_lib.dump(creds, f)
    print(f"✓ Wrote credentials to {creds_path}")
else:
    print("⚠️  No OAuth token - skipping credentials file")

# Environment snapshot
env_snapshot = {
    'workspace_path': WORKSPACE_PATH,
    'python_version': sys.version.split()[0],
    'has_oauth_token': bool(oauth_token),
    'has_api_key': 'ANTHROPIC_API_KEY' in os.environ,
}
try:
    import torch
    env_snapshot['gpu'] = torch.cuda.get_device_name(0) if torch.cuda.is_available() else False
except:
    env_snapshot['gpu'] = False

for tool in ['claude', 'black', 'isort', 'socat', 'bwrap']:
    r = subprocess.run(f'which {tool}', shell=True, capture_output=True)
    env_snapshot[f'has_{tool}'] = r.returncode == 0

with open(f"{WORKSPACE_PATH}/ENVIRONMENT.json", 'w') as f:
    json_lib.dump(env_snapshot, f, indent=2)

print("\nEnvironment:")
print(json_lib.dumps(env_snapshot, indent=2))


In [ ]:
#@title Write Skills and Config
print("=" * 60)
print("STEP 4: Write Skills & Config")
print("=" * 60)

# Create skills directory
skills_dir = f"{CLAUDE_CONFIG_PATH}/skills"
os.makedirs(skills_dir, exist_ok=True)

# Skill: IPYNB Editor
ipynb_skill = '''# IPYNB Notebook Skill

You are an expert at creating and editing Jupyter/Colab notebooks (.ipynb files).

## IPYNB Structure

Notebooks are JSON with this structure:
```json
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {"colab": {"provenance": []}, "kernelspec": {...}},
  "cells": [{"cell_type": "code|markdown", "source": [...], "metadata": {...}}]
}
```

## Cell Types

- `code`: Python code, `source` is list of strings (lines)
- `markdown`: Documentation, supports HTML in Colab

## Colab-Specific Features

- Form fields: `#@param {type:"string"}` after variable
- Collapsible: `#@title Section Name`
- Hide code: `#@markdown` for markdown-only display

## Best Practices

1. Each cell should do ONE thing
2. Add markdown cells explaining what comes next
3. Use `#@title` for form-style cells
4. Print progress/status for long operations
5. Handle errors gracefully with try/except
6. Use `subprocess.run()` not `!` for programmatic shell commands

## Editing Notebooks

To edit: load JSON, modify cells list, write JSON.
Use `json.dump(nb, f, indent=2)` for readable output.
'''
with open(f"{skills_dir}/ipynb-editor.md", 'w') as f:
    f.write(ipynb_skill)
print("✓ ipynb-editor skill")

# Skill: Skill Builder
skill_builder = '''# Skill Builder Skill

You create Claude Code skills - markdown files that extend Claude's capabilities.

## Skill File Structure

Skills are `.md` files in `.claude/skills/` with:
1. A clear title (# heading)
2. Context about when to use the skill
3. Specific instructions, patterns, or templates
4. Examples where helpful

## Skill Locations

- Project: `.claude/skills/skill-name.md`
- User: `~/.claude/skills/skill-name.md`

## Good Skills

- Are focused on ONE capability
- Include concrete examples
- Specify file formats, patterns, conventions
- Mention common pitfalls to avoid

## Example Skill Template

```markdown
# [Skill Name]

You are an expert at [domain].

## When to Use
[Triggers for this skill]

## Key Concepts
[Important background]

## Patterns
[Code/text patterns to follow]

## Examples
[Concrete examples]

## Pitfalls
[What to avoid]
```
'''
with open(f"{skills_dir}/skill-builder.md", 'w') as f:
    f.write(skill_builder)
print("✓ skill-builder skill")

# Skill: Notebook Customizer
customizer_skill = '''# Notebook Customizer Skill

You help users customize their Claude Code Colab bootstrap notebook.

## Capabilities

1. **Interview users** about their projects and preferences
2. **Clone repos/gists** into the workspace
3. **Analyze uploaded notebooks** to understand user patterns
4. **Modify settings** in bootstrap_config.json
5. **Create custom commands** and skills based on user needs

## Key Files

- `bootstrap_config.json` - Editable configuration
- `ENVIRONMENT.json` - Runtime info (read-only)
- `.claude/settings.json` - Claude Code settings
- `.claude/skills/` - Custom skills directory
- `.claude/commands/` - Custom slash commands

## Notebook Locations

- **Current running notebook**: Cannot directly edit the running .ipynb
  - Instead: Output code/instructions for user to paste, or
  - Create a NEW notebook file they can download and replace
  
- **User-uploaded notebooks**: In `/content/` or workspace
  - Can read, analyze, and modify these directly
  
- **Creating new notebooks**: Write to workspace, user downloads

## Customization Workflow

1. Ask what the user wants to build/do
2. Check ENVIRONMENT.json for current setup
3. Suggest relevant customizations
4. Create/modify skills, commands, or config
5. If notebook changes needed, create new .ipynb file

## Example Interview Questions

- What kind of projects will you work on? (ML training, data analysis, web dev)
- Do you have existing repos to clone?
- What tools/libraries do you use most?
- Any specific workflows or commands you repeat often?
'''
with open(f"{skills_dir}/notebook-customizer.md", 'w') as f:
    f.write(customizer_skill)
print("✓ notebook-customizer skill")

# Write minimal guide
guide = '''# Claude Code Colab Debug Environment

## Files
- ENVIRONMENT.json - Runtime snapshot
- bootstrap_config.json - Editable config

## Skills Available
- ipynb-editor - Create/edit Jupyter notebooks
- skill-builder - Create new skills
- notebook-customizer - Customize this environment

## Slash Commands
- /cost - Token usage
- /clear - Reset context
- /compact - Reduce context size
'''
with open(f"{WORKSPACE_PATH}/CLAUDE_CODE_COLAB_GUIDE.md", 'w') as f:
    f.write(guide)
shutil.copy(f"{WORKSPACE_PATH}/CLAUDE_CODE_COLAB_GUIDE.md", os.path.expanduser("~/CLAUDE_CODE_COLAB_GUIDE.md"))
print("✓ guide")

# Config
config = {'project_name': PROJECT_NAME, 'debug': True}
with open(f"{WORKSPACE_PATH}/bootstrap_config.json", 'w') as f:
    json_lib.dump(config, f, indent=2)
print("✓ config")

print(f"\nSkills: {os.listdir(skills_dir)}")


In [ ]:
#@title Pre-Claude Diagnostics
print("=" * 60)
print("STEP 5: Diagnostics")
print("=" * 60)

tests = [
    ("Claude binary", "ls -la ~/.local/bin/claude"),
    ("Claude version", "~/.local/bin/claude --version"),
    ("TERM", "echo $TERM"),
    ("Sandbox deps", "which socat bwrap"),
    ("Auth env vars", "env | grep -E '(ANTHROPIC|CLAUDE)' | sed 's/=.*/=***/' "),
    ("Credentials file", "ls -la ~/.claude/.credentials.json 2>/dev/null || echo 'No credentials yet'"),
    ("Skills", "ls ~/.claude/skills/"),
]

for name, cmd in tests:
    result = subprocess.run(f"source ~/.bashrc 2>/dev/null; {cmd}", 
                          shell=True, capture_output=True, text=True, executable='/bin/bash')
    output = (result.stdout + result.stderr).strip()
    print(f"\n[{name}]")
    print(f"  {output}")


In [ ]:
#@title Test Claude Code Headless
print("="*60)
print("STEP 6: Claude Code Headless Test")
print("="*60)

DIAGNOSTIC_PROMPT = '''Run a quick diagnostic:

1. Read ENVIRONMENT.json and confirm you can see it
2. List the skills in .claude/skills/ 
3. List the agents in .claude/agents/
4. Run: echo "test" to verify bash works
5. Report any issues

Be concise.
'''

print("\nRunning Claude with diagnostic prompt...")
print("-"*40)

import subprocess
cmd = f'''cd "{WORKSPACE_PATH}" && claude -p "{DIAGNOSTIC_PROMPT}" --output-format text 2>&1'''
full_cmd = f"source ~/.bashrc; {cmd}"

try:
    result = subprocess.run(
        full_cmd,
        shell=True,
        capture_output=True,
        text=True,
        executable='/bin/bash',
        timeout=120
    )
    print("STDOUT:")
    print(result.stdout if result.stdout else "(empty)")
    if result.stderr:
        print("\nSTDERR:")
        print(result.stderr)
    print(f"\nReturn code: {result.returncode}")
except subprocess.TimeoutExpired:
    print("⚠️ Command timed out after 120s")
except Exception as e:
    print(f"⚠️ Error: {e}")


In [ ]:
#@title Extract OAuth Token (after manual auth)
print("=" * 60)
print("OAUTH TOKEN EXTRACTION")
print("=" * 60)

creds_path = os.path.expanduser("~/.claude/.credentials.json")
if os.path.exists(creds_path):
    with open(creds_path, 'r') as f:
        creds = json_lib.load(f)
    print("\n✓ Credentials found!")
    print("\nTo reuse this auth, add to Colab Secrets (key icon in sidebar):")
    print(f"\n  Name: CLAUDE_OAUTH_TOKEN")
    if 'accessToken' in creds:
        token = creds['accessToken']
        print(f"  Value: {token[:20]}...{token[-10:]}")
        print(f"\n  (Full token is {len(token)} chars)")
        print(f"\n  Full token for copy:")
        print(f"  {token}")
    else:
        print(f"  Creds structure: {list(creds.keys())}")
else:
    print("\n✗ No credentials file yet.")
    print("\nTo authenticate:")
    print("1. Open terminal")
    print("2. Run: source ~/.bashrc && cd /content/claude-workspaces/debug-test && claude")
    print("3. Complete OAuth flow")
    print("4. Re-run this cell to extract token")


In [ ]:
#@title Summary & Download Log
print("=" * 60)
print("DEBUG COMPLETE")
print("=" * 60)
print(f"\nLog file: {LOG_FILE}")
print("Download it from the Files panel (folder icon) or run:")
print("  from google.colab import files; files.download('/content/debug_log.txt')")

# Also show file size
import os
size = os.path.getsize(LOG_FILE)
print(f"\nLog size: {size} bytes")
